Usaremos HuggingFace

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
hf_key = os.environ.get('HUGGINGFACEHUB_API_TOKEN')

os.environ['HUGGINGFACEHUB_API_TOKEN'] = hf_key

In [2]:
from langchain import PromptTemplate

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

# user question
question = "What is the capital city of France?"

In [3]:
from langchain import HuggingFaceHub, LLMChain

# initialize Hub LLM
hub_llm = HuggingFaceHub(
        repo_id='google/flan-t5-large',
    model_kwargs={'temperature':0}
)

# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

# ask the user question about the capital of France
print(llm_chain.invoke(question))

ValidationError: 1 validation error for HuggingFaceHub
__root__
  Got invalid task fill-mask, currently only dict_keys(['translation', 'summarization', 'conversational', 'text-generation', 'text2text-generation']) are supported (type=value_error)

Multiple questions

In [10]:
qa = [
    {'question': "What is the capital city of France?"},
    {'question': "What is the largest mammal on Earth?"},
    {'question': "Which gas is most abundant in Earth's atmosphere?"},
    {'question': "What color is a ripe banana?"}
]
res = llm_chain.generate(qa)
print( res )

generations=[[Generation(text='paris')], [Generation(text='giraffe')], [Generation(text='nitrogen')], [Generation(text='yellow')]] llm_output=None run=[RunInfo(run_id=UUID('9c30e640-b91d-420c-ac2d-bb1ea280db71')), RunInfo(run_id=UUID('444573c4-9232-460c-96f3-eb3f2ca67471')), RunInfo(run_id=UUID('0223a431-82ed-42c4-8166-2f64039a4e5f')), RunInfo(run_id=UUID('ff6b1ea6-3728-4fdd-97d3-16317451e129'))]


We can modify our prompt template to include multiple questions to implement a second approach. The language model will understand that we have multiple questions and answer them sequentially. This method performs best on more capable models.

In [16]:
from langchain_openai import OpenAI

opai_llm = OpenAI(name="gpt-3.5-turbo", temperature=0)

multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=opai_llm
)

qs_str = (
    "What is the capital city of France?\n" +
    "What is the largest mammal on Earth?\n" +
    "Which gas is most abundant in Earth's atmosphere?\n" +
		"What color is a ripe banana?\n"
)
ans = llm_chain.run(qs_str)
print(ans)


1. The capital city of France is Paris.
2. The largest mammal on Earth is the blue whale.
3. The gas most abundant in Earth's atmosphere is nitrogen.
4. A ripe banana is typically yellow.


Text Summarization

In [21]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(name="gpt-3.5-turbo", temperature=0)

Next, we define a prompt template for summarization:

In [22]:
summarization_template = "Summarize the following text to one sentence: {text}"
summarization_prompt = PromptTemplate(input_variables=["text"], template=summarization_template)
summarization_chain = LLMChain(llm=llm, prompt=summarization_prompt)

In [23]:
text = """LangChain provides many modules that can be used to build language model applications.
Modules can be combined to create more complex applications, or be used individually for simple applications.
The most basic building block of LangChain is calling an LLM on some input. Let’s walk through a simple example
of how to do this. For this purpose, let’s pretend we are building
a service that generates a company name based on what the company makes."""

summarized_text = summarization_chain.predict(text=text)

In [24]:
summarized_text

'\n\nLangChain offers various modules for building language model applications, which can be used together or separately, with the most basic being calling an LLM on input, as demonstrated in a simple example of creating a company name based on its product.'

Text Translation

In [26]:
translation_template = "Translate the following text from {source_language} to {target_language}: {text}"
translation_prompt = PromptTemplate(input_variables=["source_language", "target_language", "text"], template=translation_template)
translation_chain = LLMChain(llm=llm, prompt=translation_prompt)

In [28]:
source_language = "English"
target_language = "Spanish"
text = "My dog is crazy"
translated_text = translation_chain.predict(source_language=source_language, target_language=target_language, text=text)
print(translated_text)



Mi perro está loco.
